<a href="https://colab.research.google.com/github/DonRoboto/AprendizajeAutomatizado/blob/master/Tarea3/Tarea3_ej3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [0]:
def sigmoide(X, weight):
    z = np.dot(X, weight)
    return 1 / (1 + np.exp(-z))

def funcion_perdida(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

def descenso_gradiente(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]

def actualiza_pesos(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

In [0]:
def fit_modelo(X_train, y_train):
  num_iter = 100000

  intercept_train = np.ones((X_train.shape[0], 1)) 
  X_train = np.concatenate((intercept_train, X_train), axis=1)
  theta = np.zeros(X_train.shape[1])

  for i in range(num_iter):
    h = sigmoide(X_train, theta)
    gradient = descenso_gradiente(X_train, h, y_train)
    theta = actualiza_pesos(theta, 0.001, gradient)
  return theta   

In [0]:
def evalua_modelo(X_test, theta):
  intercept_test = np.ones((X_test.shape[0], 1)) 
  X_test = np.concatenate((intercept_test, X_test), axis=1)
  y_val = sigmoide(X_test, theta)
  
  y_label = [0] * len(y_val)
  v_max = max(y_val)
  v_min = min(y_val)
  umbral = (v_max-v_min)/2

  for i in range(len(y_val)):
    if(y_val[i]>umbral):
        y_label[i]=1
    else:
        y_label[i]=0
  return y_label

In [0]:
df_train =  pd.read_csv("https://raw.githubusercontent.com/DonRoboto/AprendizajeAutomatizado/master/breast-cancer-wisconsin.data",header=None, delimiter=",")
df_train[6] = df_train[6].map({'?': 0, '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10})
df_train[10] = df_train[10].map({2: 1, 4: 0})

df_train[:] = df_train[:].astype(float)
data_train = df_train.to_numpy()

X_dataP = data_train[:, 1:9]
y_dataP = data_train[:, -1]


In [6]:
score_final = []
for ex in range(10):

  X_data, y_data = shuffle(X_dataP, y_dataP)

  kf = StratifiedKFold(n_splits=5)
  pred_test_full =0
  
  for train_index,test_index in kf.split(X_data,y_data):
    cv_score =[]
    xtr,xvl = X_data[train_index],X_data[test_index]
    ytr,yvl = y_data[train_index],y_data[test_index]
   
    theta = fit_modelo(xtr, ytr)
    y_hat = evalua_modelo(xvl, theta)
    score = roc_auc_score(yvl,y_hat)
  
    cv_score.append(score)
  
  print(ex, " ", score.mean())   
  score_final.append(score.mean())   
print("Resultado Regresión Logística:", sum(score_final)/len(score_final))   


0   0.9479166666666667
1   0.9626831501831503
2   0.9632554945054945
3   0.9621108058608058
4   0.9571886446886448
5   0.9302884615384616
6   0.9210164835164836
7   0.9467719780219781
8   0.9676053113553114
9   0.9632554945054945
Resultado Regresión Logística: 0.9522092490842491


In [7]:
score_final = []
for ex in range(10):

  X_data, y_data = shuffle(X_dataP, y_dataP)

  kf = StratifiedKFold(n_splits=5)
  pred_test_full =0
  
  for train_index,test_index in kf.split(X_data,y_data):
    cv_score =[]
    xtr,xvl = X_data[train_index],X_data[test_index]
    ytr,yvl = y_data[train_index],y_data[test_index]
    
    lr = GaussianNB()
    lr.fit(xtr,ytr)
    score = roc_auc_score(yvl,lr.predict(xvl))
  
    cv_score.append(score)
  
  print(ex, " ", score.mean())   
  score_final.append(score.mean())   
print("Resultado Clasificador Bayesiano Ingenuo:", sum(score_final)/len(score_final)) 

0   0.9785943223443223
1   0.9945054945054945
2   0.989010989010989
3   0.9626831501831503
4   0.9566163003663003
5   0.9626831501831503
6   0.9676053113553114
7   0.9791666666666667
8   0.956043956043956
9   0.96875
Resultado Clasificador Bayesiano Ingenuo: 0.9715659340659342
